# Langchain

In [26]:
# !pip -q install openai langchain huggingface_hub
!pip uninstall langchain -y
!pip install langchain
!pip install -qU langchain-openai
!python --version
!pip install langchain-openai
!pip install -U langchain-ollama
!pip install -qU langchain_community beautifulsoup4
!pip install faiss-cpu

Found existing installation: langchain 0.3.14
Uninstalling langchain-0.3.14:
  Successfully uninstalled langchain-0.3.14
  Using cached langchain-0.3.14-py3-none-any.whl.metadata (7.1 kB)
Using cached langchain-0.3.14-py3-none-any.whl (1.0 MB)
Python 3.11.11
  Using cached ollama-0.4.5-py3-none-any.whl.metadata (4.7 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
Using cached ollama-0.4.5-py3-none-any.whl (13 kB)
Using cached httpx-0.27.2-py3-none-any.whl (76 kB)
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [5]:
# from langgraph.graph import StateGraph,START,END
# from langgraph.graph.message import add_messages

# from typing_extensions import TypedDict
# from typing import Annotated

# from IPython.display import Image, display
# !python --version

Python 3.10.16


In [17]:
import os


In [1]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM


In [2]:
template = """Question: {question}

Answer: let me explain it step by step"""

prompt = ChatPromptTemplate.from_template(template)
prompt

prompt1 = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a ufc fighter, explain how to perform the following move"),
        ("human","{input}")
    ]
)

In [3]:
model = OllamaLLM(model="llama3.2")

chain = prompt1 | model



In [56]:
res=chain.invoke({"input": "caf kick"})

In [59]:
print(res)


The Caf Kick. It's a classic Muay Thai technique that can be devastating in the right hands.

First, you need to understand the basics of Muay Thai and how it applies to the Cafe Kick. The Cafe Kick is a type of roundhouse kick that targets the knee area. To perform this move effectively, follow these steps:

**Setup**

1. Begin by standing with your feet shoulder-width apart, with your dominant foot forward (for most people, this means right foot). Keep your knees slightly bent and your weight evenly distributed between both feet.
2. Position yourself at an angle to the target, with your lead leg pointing towards the opponent's knee area.

**Execution**

1. Start by rotating your hips and generating power from your core. This will help you generate torque for the kick.
2. As you begin to rotate, extend your back leg (the one behind you) and transfer your weight onto it. Keep your front leg straight and your foot flexed.
3. Use your lead leg to drive the kick, keeping it in a 45-degree

# doc loader

In [22]:
!pip install python-dotenv

In [26]:

from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()
LANGSMITH_TRACING = os.getenv('LANGSMITH_TRACING')
LANGSMITH_ENDPOINT = os.getenv('LANGSMITH_ENDPOINT')
LANGSMITH_API_KEY = os.getenv('LANGSMITH_API_KEY')


def text_to_embeddings(docs):
    embeddings_model = OllamaEmbeddings(model="llama3.2") 
    embeddings = [embeddings_model.embed_query(chunk.page_content) for chunk in docs]

def web_doc_loader(url:str,chunk_size=1000,chunk_overlap=0):
    
    loader = WebBaseLoader(url)
    docs1=loader.load()
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    split_docs=splitter.split_documents(docs1)
    print(f"Number of chunks: {len(split_docs)}")

    # Initialize Ollama embeddings
   
     # Generate embeddings for each chunk
    

    # Optionally, return both the chunks and their embeddings
    return split_docs
    

url="https://www.apple.com"
split_docs = web_doc_loader(url)
embeddings=text_to_embeddings(split_docs)
split_docs[:2]

Number of chunks: 8


[Document(metadata={'source': 'https://www.apple.com', 'title': 'Apple', 'language': 'en-US'}, page_content='Apple\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nApple\n\nAppleStoreMaciPadiPhoneWatch\nVisionAirPodsTV & HomeEntertainmentAccessoriesSupport\n\n\n0+\n\n\n\n\n\n\n\n\xa0\n\niPhone 16 Pro\nHello, Apple\xa0Intelligence.\n\nLearn more\nBuy\n\n\n\n\n\n\n\n\n\n\n\n\n\xa0\n\niPhone 16\nHello, Apple\xa0Intelligence.\n\nLearn more\nBuy\n\n\n\n\n\n\n\n\n\n\n\n\n\xa0\n\nApple Watch Series 10\nThinstant classic.\n\nLearn more\nBuy\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\xa0\n\nApple Fitness+\nFitness for everyone. Personalized for you.\nGet up to 3 months on us.1\n\nLearn more\nTry it free\n\n\n\n\n\n\n\n\n\n\n\n\n\xa0\n\n\niPad Air\nTwo sizes. Faster chip. Does it all.\n\nLearn more\nBuy\n\n\n\nHello, Apple Intelligence.\n\n\n\n\n\n\n\n\n\n\n\n\n\xa0\n\n\nMacBook Pro\nA work of smart.\n\nLearn more\nBuy\n\n\n\nHello, Apple Intelligence.\n\n\n\n\n\n\n\n\n\n\n\n\n\xa0\n\nApple Watch Ultra 2\nNew finish. Ne

In [27]:
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import MessagesPlaceholder
def to_vector_store(docs):

    embeddings_model = OllamaEmbeddings(model="llama3.2")  
    vector_store = FAISS.from_documents(docs, embeddings_model)
    return vector_store

def create_chain(vector_store):

    # template = """Answer the following question of the user only from the context
    # Context:{context},
    # Question: {input}"""

    promptdoc = ChatPromptTemplate.from_messages([
        ("system","Answer the following question only using the {context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human","{input}")
    ])

    # promptdoc = ChatPromptTemplate.from_template(template)
    model = OllamaLLM(model="llama3.2")
    chain = create_stuff_documents_chain(llm=model,prompt=promptdoc)
    
    retriever = vector_store.as_retriever()
    retreival_chain=create_retrieval_chain(retriever,chain)
    return retreival_chain

def process_chat(retreival_chain,question,chat_history):
    
    res = retreival_chain.invoke({
    "chat_history":chat_history,
    "input": question
    })
    return res["answer"]
    

    
vector_store=to_vector_store(split_docs)
retreival_chain=create_chain(vector_store)

# process_chat(retreival_chain,"whats the specs of the iphone")




# following input

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage
chat_history=[

]

while True:
    inp=input("\n you->")
    if inp.lower()=="q":
        print("Goodbye")
        break
    try:
        
        response = process_chat(retreival_chain,inp,chat_history)
        chat_history.append(HumanMessage(content=inp))
        chat_history.append(AIMessage(content=response))
        print("\n Asistant->"+response)
    except Exception as e:
        print(f"Error2: {e}")


 you-> hello



 Asistant->Hello! It seems like you're looking for information about Apple Card or trade-in options. Is there something specific you'd like to know?


In [31]:
print(len(split_docs), len(res['context']))

8 4


In [77]:
#chain = promptdoc | model
chain = create_stuff_documents_chain(llm=model,prompt=promptdoc)
res=chain.invoke({"question": "specs of the new iphone","context":ap_docs})

In [86]:

ap_docs = web_doc_loader(url)
template = """Answer the following question of the user only from the context
Context:{context},
Question: {question}"""

promptdoc = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="llama3.2")

chain = promptdoc | model

res=chain.invoke({"question": "specs of the new iphone","context":[ap_docs]})